#Team Armadillo — Enterprise Ready AI
IBM AI Experiential Learning Lab 2025  

This notebook demonstrates the end-to-end pipeline for policy risk evaluation using watsonx.ai Granite models.

In [ ]:
# ================================================================
# SECTION 1 — SETUP
# ================================================================
import os, json, requests, pandas as pd
from scripts.watsonx_connect import get_iam_token
from scripts.log_governance import log_governance

IBM_API_KEY = "YOUR_API_KEY"  # Replace with your IBM Cloud API key
ACCESS_TOKEN = get_iam_token(IBM_API_KEY)

In [ ]:
# ================================================================
# SECTION 2 — LOAD DATA
# ================================================================
data_path = "../data/sample_bank_policy.csv"
if os.path.exists(data_path):
    df = pd.read_csv(data_path)
    print(f"Loaded policy data: {df.shape[0]} records")
else:
    print("No dataset found — using mock data.")
    df = pd.DataFrame({"Policy": ["Loan Eligibility", "Default Mitigation"]})

In [ ]:
# ================================================================
# SECTION 3 — GRANITE MODEL INFERENCE
# ================================================================
endpoint = "https://us-south.ml.cloud.ibm.com/ml/v1/text/generation"
headers = {"Authorization": f"Bearer {ACCESS_TOKEN}", "Content-Type": "application/json"}

def query_granite(prompt, model_id="ibm-granite-13b-instruct"):
    payload = {
        "model_id": model_id,
        "input": prompt,
        "parameters": {"decoding_method": "greedy", "max_new_tokens": 150, "temperature": 0.3}
    }
    r = requests.post(endpoint, headers=headers, json=payload)
    r.raise_for_status()
    return r.json()["results"][0]["generated_text"]

prompt = "Summarize risk considerations and compliance obligations in the bank policy."
summary = query_granite(prompt)
print("📘 Model Output:\n", summary)

In [ ]:
# ================================================================
# SECTION 4 — RISK EVALUATION
# ================================================================
def evaluate_risk(text):
    keywords = ["default", "fraud", "liability", "threshold", "compliance"]
    hits = sum(1 for w in keywords if w in text.lower())
    return round(hits / len(keywords), 2)

risk_score = evaluate_risk(summary)
print(f"🏦 Derived Risk Score: {risk_score}")

In [ ]:
# ================================================================
# SECTION 5 — GOVERNANCE LOGGING
# ================================================================
record = {"summary": summary, "risk_score": risk_score}
log_path = log_governance(record)
print(f"✅ Log created: {log_path}")